In [1]:
import itertools 
import matplotlib 
import matplotlib.style 
import numpy as np 
import sys 
import random 
import math
import pandas as pd

In [2]:
# Parameters
n = 3 # number of competitors
delta = 0.95 # discount factor
training_years = 3 # number of years using training data
years = 7 # number of years
weeks = 52 # weeks per year
T = years*weeks # total time periods
t = training_years*weeks # training periods
J = 1000000 # offline training
epi = 10 # number of episodes
a = 0.2 # learning parameter
beta = 1-(0.0001**(1/J))
print(beta)

# Parameters for calculating market shares
alpha = -0.1312 # price parameter (NL)
rho = 0.6299 # nesting parameter (NL)

# Markups
# 1: Bud Light, 2: Coors Light, 3: Miller Lite
Markups = [3.52, 2.36, 2.78]


9.210297956974145e-06


In [3]:
# Defining action set

p99 = 10.678
p50 = 8.980137
p1 = 7.971982

actions_pre_low = np.linspace(p1, p50, num=10)
actions_low = actions_pre_low.tolist()
actions_pre_high = np.linspace(p50, p99, num=10)
actions_high = actions_pre_high.tolist()

actions = actions_low + actions_high
actions=list(set(actions))

for i in range(len(actions)):
    actions[i]=round(actions[i],6)
    
print(actions)

A=len(actions)

[7.971982, 8.083999, 8.196016, 8.420051, 8.308034, 8.532068, 8.644085, 8.756103, 8.86812, 8.980137, 9.168788, 9.35744, 10.112046, 9.546091, 9.734743, 9.923394, 10.300697, 10.489349, 10.678]


In [4]:
# Defining states

states = list(itertools.product(actions, actions, actions))
print(len(states))

S=len(states)

6859


In [5]:
# Getting list of pre prices


Bud_Pre = pd.read_stata('Bud_Pre.dta')
Coors_Pre = pd.read_stata('Coors_Pre.dta')
Miller_Pre = pd.read_stata('Miller_Pre.dta')

Bud_Pre_Prices=[]
Coors_Pre_Prices=[]
Miller_Pre_Prices=[]

for i, j in Bud_Pre.iterrows():
    Bud_Pre_Prices.append(j[0])
    
for i, j in Coors_Pre.iterrows():
    Coors_Pre_Prices.append(j[0])
    
for i, j in Miller_Pre.iterrows():
    Miller_Pre_Prices.append(j[0])

In [6]:
# Getting list of closest actions to observed prices

Bud_Prices_Pre=[]

for i in range(len(Bud_Pre_Prices)):
    k = min(actions, key=lambda x:abs(x-Bud_Pre_Prices[i]))
    Bud_Prices_Pre.append(k)

Coors_Prices_Pre=[]

for i in range(len(Coors_Pre_Prices)):
    k = min(actions, key=lambda x:abs(x-Coors_Pre_Prices[i]))
    Coors_Prices_Pre.append(k)
    
Miller_Prices_Pre=[]

for i in range(len(Miller_Pre_Prices)):
    k = min(actions, key=lambda x:abs(x-Miller_Pre_Prices[i]))
    Miller_Prices_Pre.append(k)

In [7]:
# Getting outside deltas

Outside_Post = pd.read_stata('Outside_Post.dta')

Outside_Deltas=[]

for i, j in Outside_Post.iterrows():
    Outside_Deltas.append((j[2]))

In [8]:
# Shares function

def expdelta(price):
    return math.exp((alpha*price)/(1-rho))

def shares(p1, p2, p3, out):
    return expdelta(p1)/(expdelta(p1)+expdelta(p2)+expdelta(p3) + out)

def profits(p1, p2, p3, mc, out):
    return (p1-mc)*shares(p1, p2, p3, out)

In [9]:
# Getting list of pre-profits

Bud_Pre_Profits = pd.read_stata('Bud_Pre_Profits.dta')
Coors_Pre_Profits = pd.read_stata('Coors_Pre_Profits.dta')
Miller_Pre_Profits = pd.read_stata('Miller_Pre_Profits.dta')

Bud_Pre_Prof=[]
Coors_Pre_Prof=[]
Miller_Pre_Prof=[]

for i, j in Bud_Pre_Profits.iterrows():
    Bud_Pre_Prof.append(j[1])
    
for i, j in Coors_Pre_Profits.iterrows():
    Coors_Pre_Prof.append(j[1])
    
for i, j in Miller_Pre_Profits.iterrows():
    Miller_Pre_Prof.append(j[1])

In [10]:
# Q-matrices

# Bud Light (ABI)
q_init_abi = np.zeros([S,A])

# Coors Light (Coors)
q_init_coors = np.zeros([S,A])

# Miller Lite (Miller)
q_init_miller = np.zeros([S,A])

# Building "initial" Q-matrices
for i in range(t-1):
    # Actions
    action_bud_pre=Bud_Prices_Pre[i+1]
    action_bud=actions.index(action_bud_pre)
    action_coors_pre=Coors_Prices_Pre[i+1]
    action_coors=actions.index(action_coors_pre)
    action_miller_pre=Miller_Prices_Pre[i+1]
    action_miller=actions.index(action_miller_pre)
    
    # States
    state_pre=(Bud_Prices_Pre[i], Coors_Prices_Pre[i], Miller_Prices_Pre[i])
    state=states.index(state_pre)
    
    next_state_pre=(Bud_Prices_Pre[i+1], Coors_Prices_Pre[i+1], Miller_Prices_Pre[i+1])
    next_state=states.index(next_state_pre)
    
    # Profits
    bud_profit=Bud_Pre_Prof[i+1]
    coors_profit=Coors_Pre_Prof[i+1]
    miller_profit=Miller_Pre_Prof[i+1]
    
    # Q-matrix updates
    q_init_abi[state, action_bud] = (1-a)*q_init_abi[state, action_bud] + a*(bud_profit + delta*np.max(q_init_abi[next_state]))
    q_init_coors[state, action_coors] = (1-a)*q_init_coors[state, action_coors] + a*(coors_profit + delta*np.max(q_init_coors[next_state]))
    q_init_miller[state, action_miller] = (1-a)*q_init_miller[state, action_miller] + a*(miller_profit + delta*np.max(q_init_miller[next_state]))

    

In [11]:
# Post marginal costs (calculated from Stata)

bud_MC=5.640409
coors_MC=6.509169
miller_MC=5.818666

In [202]:
# Bud offline training

next_state_pre=random.choice(states)
next_state=states.index(next_state_pre)

for i in range(J):
    state=next_state
    epsilon=(1-beta)**t
    #print(epsilon)

    #Actions  
    if epsilon>random.uniform(0,1):
        action_bud_pre = random.sample(actions,1)[0]
        action_bud = actions.index(action_bud_pre)
        
    else:
        action_bud = np.argmax(q_init_abi[state])
        action_bud_pre=actions[action_bud]
 

    
    action_coors_pre = random.sample(actions,1)[0]
    action_coors = actions.index(action_coors_pre)

    action_miller_pre = random.sample(actions,1)[0]
    action_miller = actions.index(action_miller_pre)



    #Next State
    next_state_pre=(action_bud_pre, action_coors_pre, action_miller_pre)

    next_state = states.index(next_state_pre)


    #Profits
    bud_profit=profits(action_bud_pre, action_coors_pre, action_miller_pre, bud_MC)
    #coors_profit=profits(action_coors_pre, action_bud_pre, action_miller_pre, coors_MC)
    #miller_profit=profits(action_miller_pre, action_coors_pre, action_bud_pre, miller_MC)

    # Q-matrix updates
    q_init_abi[state, action_bud] = (1-a)*q_init_abi[state, action_bud] + a*(bud_profit + delta*np.max(q_init_abi[next_state]))
    #q_init_coors[state, action_coors] = (1-a)*q_init_coors[state, action_coors] + a*(coors_profit + delta*np.max(q_init_coors[next_state]))
    #q_init_miller[state, action_miller] = (1-a)*q_init_miller[state, action_miller] + a*(miller_profit + delta*np.max(q_init_miller[next_state]))

    

KeyboardInterrupt: 

In [ ]:
# Coors offline training

next_state_pre=random.choice(states)
next_state=states.index(next_state_pre)

for i in range(J):
    state=next_state
    epsilon=(1-beta)**t
    #print(epsilon)

    #Actions  
    if epsilon>random.uniform(0,1):
        action_coors_pre = random.sample(actions,1)[0]
        action_coors = actions.index(action_coors_pre)

    else:
        action_coors = np.argmax(q_init_coors[state])
        action_coors_pre=actions[action_coors]


    
    action_bud_pre = random.sample(actions,1)[0]
    action_bud = actions.index(action_bud_pre)

    action_miller_pre = random.sample(actions,1)[0]
    action_miller = actions.index(action_miller_pre)



    #Next State
    next_state_pre=(action_bud_pre, action_coors_pre, action_miller_pre)

    next_state = states.index(next_state_pre)


    #Profits
    #bud_profit=profits(action_bud_pre, action_coors_pre, action_miller_pre, bud_MC)
    coors_profit=profits(action_coors_pre, action_bud_pre, action_miller_pre, coors_MC)
    #miller_profit=profits(action_miller_pre, action_coors_pre, action_bud_pre, miller_MC)

    # Q-matrix updates
    #q_init_abi[state, action_bud] = (1-a)*q_init_abi[state, action_bud] + a*(bud_profit + delta*np.max(q_init_abi[next_state]))
    q_init_coors[state, action_coors] = (1-a)*q_init_coors[state, action_coors] + a*(coors_profit + delta*np.max(q_init_coors[next_state]))
    #q_init_miller[state, action_miller] = (1-a)*q_init_miller[state, action_miller] + a*(miller_profit + delta*np.max(q_init_miller[next_state]))

    

In [162]:
# Miller offline training

next_state_pre=random.choice(states)
next_state=states.index(next_state_pre)

for i in range(J):
    state=next_state
    epsilon=(1-beta)**t
    #print(epsilon)

    #Actions  
    if epsilon>random.uniform(0,1):
        action_miller_pre = random.sample(actions,1)[0]
        action_miller = actions.index(action_miller_pre)

    else:
        action_miller = np.argmax(q_init_miller[state])
        action_miller_pre=actions[action_miller]
        

    
    action_coors_pre = random.sample(actions,1)[0]
    action_coors = actions.index(action_coors_pre)

    action_bud_pre = random.sample(actions,1)[0]
    action_bud = actions.index(action_bud_pre)



    #Next State
    next_state_pre=(action_bud_pre, action_coors_pre, action_miller_pre)

    next_state = states.index(next_state_pre)


    #Profits
    #bud_profit=profits(action_bud_pre, action_coors_pre, action_miller_pre, bud_MC)
    #coors_profit=profits(action_coors_pre, action_bud_pre, action_miller_pre, coors_MC)
    miller_profit=profits(action_miller_pre, action_coors_pre, action_bud_pre, miller_MC)

    # Q-matrix updates
    #q_init_abi[state, action_bud] = (1-a)*q_init_abi[state, action_bud] + a*(bud_profit + delta*np.max(q_init_abi[next_state]))
    #q_init_coors[state, action_coors] = (1-a)*q_init_coors[state, action_coors] + a*(coors_profit + delta*np.max(q_init_coors[next_state]))
    q_init_miller[state, action_miller] = (1-a)*q_init_miller[state, action_miller] + a*(miller_profit + delta*np.max(q_init_miller[next_state]))

    

In [163]:
print(q_init_abi)

[[3.15324847 3.27390734 2.50105669 ... 3.3494383  2.4956488  1.93499246]
 [2.45080435 3.14267298 2.92020407 ... 3.22126985 3.10199393 2.80476956]
 [2.81346644 0.98778418 2.8126845  ... 2.37833116 3.31642413 2.51173434]
 ...
 [3.22920323 2.23659693 2.81947179 ... 1.12386997 3.97498162 2.60353677]
 [2.45671994 2.64241338 2.29056401 ... 2.96337312 2.5583901  2.85134682]
 [2.23035928 2.95349664 2.09508228 ... 3.00668943 3.27243398 2.96966772]]


In [12]:
Bud_Prices_Outer=[]
Bud_Explore_Outer=[]
Coors_Prices_Outer=[]
Coors_Explore_Outer=[]
Miller_Prices_Outer=[]
Miller_Explore_Outer=[]

for j in range(epi):
    next_state_pre=random.choice(states)
    next_state=states.index(next_state_pre)

    for i in range(J):
        state=next_state
        epsilon=(1-beta)**t
        outside=Outside_Deltas[i]
        #print(epsilon)

        #Actions  
        if epsilon>random.uniform(0,1):
            action_miller_pre = random.sample(actions,1)[0]
            action_miller = actions.index(action_miller_pre)

        else:
            action_miller = np.argmax(q_init_miller[state])
            action_miller_pre=actions[action_miller]



        action_coors_pre = random.sample(actions,1)[0]
        action_coors = actions.index(action_coors_pre)

        action_bud_pre = random.sample(actions,1)[0]
        action_bud = actions.index(action_bud_pre)



        #Next State
        next_state_pre=(action_bud_pre, action_coors_pre, action_miller_pre)

        next_state = states.index(next_state_pre)


        #Profits
        #bud_profit=profits(action_bud_pre, action_coors_pre, action_miller_pre, bud_MC)
        #coors_profit=profits(action_coors_pre, action_bud_pre, action_miller_pre, coors_MC)
        miller_profit=profits(action_miller_pre, action_coors_pre, action_bud_pre, miller_MC, outside)

        # Q-matrix updates
        #q_init_abi[state, action_bud] = (1-a)*q_init_abi[state, action_bud] + a*(bud_profit + delta*np.max(q_init_abi[next_state]))
        #q_init_coors[state, action_coors] = (1-a)*q_init_coors[state, action_coors] + a*(coors_profit + delta*np.max(q_init_coors[next_state]))
        q_init_miller[state, action_miller] = (1-a)*q_init_miller[state, action_miller] + a*(miller_profit + delta*np.max(q_init_miller[next_state]))
        
    next_state_pre=random.choice(states)
    next_state=states.index(next_state_pre)

    for i in range(J):
        state=next_state
        epsilon=(1-beta)**t
        #print(epsilon)

        #Actions  
        if epsilon>random.uniform(0,1):
            action_coors_pre = random.sample(actions,1)[0]
            action_coors = actions.index(action_coors_pre)

        else:
            action_coors = np.argmax(q_init_coors[state])
            action_coors_pre=actions[action_coors]



        action_bud_pre = random.sample(actions,1)[0]
        action_bud = actions.index(action_bud_pre)

        action_miller_pre = random.sample(actions,1)[0]
        action_miller = actions.index(action_miller_pre)



        #Next State
        next_state_pre=(action_bud_pre, action_coors_pre, action_miller_pre)

        next_state = states.index(next_state_pre)


        #Profits
        #bud_profit=profits(action_bud_pre, action_coors_pre, action_miller_pre, bud_MC)
        coors_profit=profits(action_coors_pre, action_bud_pre, action_miller_pre, coors_MC, outside)
        #miller_profit=profits(action_miller_pre, action_coors_pre, action_bud_pre, miller_MC)

        # Q-matrix updates
        #q_init_abi[state, action_bud] = (1-a)*q_init_abi[state, action_bud] + a*(bud_profit + delta*np.max(q_init_abi[next_state]))
        q_init_coors[state, action_coors] = (1-a)*q_init_coors[state, action_coors] + a*(coors_profit + delta*np.max(q_init_coors[next_state]))
        #q_init_miller[state, action_miller] = (1-a)*q_init_miller[state, action_miller] + a*(miller_profit + delta*np.max(q_init_miller[next_state]))
    next_state_pre=random.choice(states)
    next_state=states.index(next_state_pre)

    for i in range(J):
        state=next_state
        epsilon=(1-beta)**t
        #print(epsilon)

        #Actions  
        if epsilon>random.uniform(0,1):
            action_bud_pre = random.sample(actions,1)[0]
            action_bud = actions.index(action_bud_pre)

        else:
            action_bud = np.argmax(q_init_abi[state])
            action_bud_pre=actions[action_bud]



        action_coors_pre = random.sample(actions,1)[0]
        action_coors = actions.index(action_coors_pre)

        action_miller_pre = random.sample(actions,1)[0]
        action_miller = actions.index(action_miller_pre)



        #Next State
        next_state_pre=(action_bud_pre, action_coors_pre, action_miller_pre)

        next_state = states.index(next_state_pre)


        #Profits
        bud_profit=profits(action_bud_pre, action_coors_pre, action_miller_pre, bud_MC, outside)
        #coors_profit=profits(action_coors_pre, action_bud_pre, action_miller_pre, coors_MC)
        #miller_profit=profits(action_miller_pre, action_coors_pre, action_bud_pre, miller_MC)

        # Q-matrix updates
        q_init_abi[state, action_bud] = (1-a)*q_init_abi[state, action_bud] + a*(bud_profit + delta*np.max(q_init_abi[next_state]))
        #q_init_coors[state, action_coors] = (1-a)*q_init_coors[state, action_coors] + a*(coors_profit + delta*np.max(q_init_coors[next_state]))
        #q_init_miller[state, action_miller] = (1-a)*q_init_miller[state, action_miller] + a*(miller_profit + delta*np.max(q_init_miller[next_state]))


    
    Bud_Post_Prices=[]
    Bud_Explore=[]
    Coors_Post_Prices=[]
    Coors_Explore=[]
    Miller_Post_Prices=[]
    Miller_Explore=[]
    next_state_pre=(Bud_Prices_Pre[t-1], Coors_Prices_Pre[t-1], Miller_Prices_Pre[t-1])
    next_state = states.index(next_state_pre)
    q_table_abi=q_init_abi
    q_table_coors=q_init_coors
    q_table_miller=q_init_miller
    for i in range(T-t):
        state=next_state

        #Actions  
        if np.mean(q_table_abi[state])==0:
            action_bud_pre = random.sample(actions,1)[0]
            action_bud = actions.index(action_bud_pre)
            Bud_Explore.append(1)
        else:
            action_bud = np.argmax(q_table_abi[state])
            action_bud_pre=actions[action_bud]
            Bud_Explore.append(0)

        if np.mean(q_table_coors[state])==0:
            action_coors_pre = random.sample(actions,1)[0]
            action_coors = actions.index(action_coors_pre)
            Coors_Explore.append(1)
        else:
            action_coors = np.argmax(q_table_coors[state])
            action_coors_pre=actions[action_coors]
            Coors_Explore.append(0)

        if np.mean(q_table_abi[state])==0:
            action_miller_pre = random.sample(actions,1)[0]
            action_miller = actions.index(action_miller_pre)
            Miller_Explore.append(1)
        else:
            action_miller = np.argmax(q_table_miller[state])
            action_miller_pre=actions[action_miller]
            Miller_Explore.append(0)


        #Next State
        next_state_pre=(action_bud_pre, action_coors_pre, action_miller_pre)

        next_state = states.index(next_state_pre)


        #Profits
        bud_profit=profits(action_bud_pre, action_coors_pre, action_miller_pre, bud_MC, outside)
        coors_profit=profits(action_coors_pre, action_bud_pre, action_miller_pre, coors_MC, outside)
        miller_profit=profits(action_miller_pre, action_coors_pre, action_bud_pre, miller_MC, outside)

        # Q-matrix updates
        q_table_abi[state, action_bud] = (1-a)*q_table_abi[state, action_bud] + a*(bud_profit + delta*np.max(q_table_abi[next_state]))
        q_table_coors[state, action_coors] = (1-a)*q_table_coors[state, action_coors] + a*(coors_profit + delta*np.max(q_table_coors[next_state]))
        q_table_miller[state, action_miller] = (1-a)*q_table_miller[state, action_miller] + a*(miller_profit + delta*np.max(q_table_miller[next_state]))

        Bud_Post_Prices.append(action_bud_pre)
        Coors_Post_Prices.append(action_coors_pre)
        Miller_Post_Prices.append(action_miller_pre)
    Bud_Prices_Outer.append(Bud_Post_Prices)
    Coors_Prices_Outer.append(Coors_Post_Prices)
    Miller_Prices_Outer.append(Miller_Post_Prices)
    Bud_Explore_Outer.append(Bud_Explore)
    Coors_Explore_Outer.append(Coors_Explore)
    Miller_Explore_Outer.append(Miller_Explore)

IndexError: list index out of range

In [13]:
print(len(Bud_Prices_Outer))
print(Bud_Prices_Outer[1])
print(Bud_Explore_Outer[1])
print(len(Bud_Prices_Outer[1]))

0


IndexError: list index out of range

In [15]:
import pickle

with open('bud_prices_T12', 'wb') as fp:
    pickle.dump(Bud_Prices_Outer, fp)

with open('bud_explore_T12', 'wb') as fp:
    pickle.dump(Bud_Explore_Outer, fp)
    
with open('coors_prices_T12', 'wb') as fp:
    pickle.dump(Coors_Prices_Outer, fp)
    
with open('coors_explore_T12', 'wb') as fp:
    pickle.dump(Coors_Explore_Outer, fp)
    
with open('miller_prices_T12', 'wb') as fp:
    pickle.dump(Coors_Prices_Outer, fp)
    
with open('miller_explore_T12', 'wb') as fp:
    pickle.dump(Miller_Explore_Outer, fp)
    


[7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 8.86812, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 8.308034, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 7.971982, 